# 1. Úvod

Současné úlohy vyžadují stále více výpočetního výkonu. Běžný osobní počítač tak nemusí pro velké jazykové modely nebo pro modely počítačové vidění(computer vision) stačit. Azure nabízí téměř neomezený výpočetní výkon, který může Váš vývoj výrazně urychlit. Vše přitom může být obsluhováno z lokálního zařízení. Tento notebook Vás seznámí se vším potřebným tak, abyste mohli rovnout začít spouštět modely na Azuru.

*Pokud si chcete vzorové příklad v tomto notebooku spouštět stáhněte si jej i se všemi lokálními soubory.*

# 2. Předpoklady

Následující řádky jsou určeny především **studentům**, s již pokročilejšími znalostmi Pythonu a strojového učení, hledající výpočetní zdroje pro náročnější problémy. Budu počítat se znalostí některého z populárních machine learning frameworků. Spustitelné příklady používají TensorFlow, ale kód je s mírnými úpravami použitelný s jakýmkoliv frameworkem.

Pro spuštění notebooku je nutné nainstalovat nasledující Python balíčky pomocí pip:

In [ ]:
!pip install azureml-core
!pip isntall tensorflow
!pip install tensorflow-datasets

# 3. Založení Azure účtu

Azure je výpočetní platforma s desítkami služeb rozličného zaměření. Můžeme zde najít služby pro hostování webových aplikací, správu IoT zařízení nebo pro nás podstatnout službu Azure Machine Learning Studio, jenž nabízí mnoho nástrojů pro vývoj modelů.


Microsoft Azure je založený na placených předplatných a kreditech. Pokud jste studenti, určitě využijte možnost si aktivovat studenstké předplatné kde po registraci dostanete kredit 100$. Skvělý návod pro to jak si ho rychle vytvořit najdete na našem STC blogu studuj.digital: https://studuj.digital/2020/01/19/aktivace-azure-for-students/.

# 4. Instalace Azure CLI

Kromě grafického rozhranní Azure Portal můžeme veškerou obsluhu Azuru provádět skrz přes command line interface (CLI). Osobně tuto metodu preferuji, jelikož ji můžu používat přímo z terminálu a po naučení jednotlivých příkazů je i rychlejší. CLI tak budeme používat i v tomto návodu pro vytváření potřebných služeb. Instalace se liší v závislosti na Vašem operačním systému.

### Windows

Pro operační systém Windows je nejjednodušší použít instalátor. Nejnovější verzi stáhnete na: https://aka.ms/installazurecliwindows

### Linux

In [ ]:
import sys

if sys.platform == "Linux":
    
    try: # pro distribuce RHEL, Fedora or CentOS
        !sudo rpm --import https://packages.microsoft.com/keys/microsoft.asc
        !echo -e "[azure-cli] name=Azure CLI baseurl=https://packages.microsoft.com/yumrepos/azure-cli enabled=1 gpgcheck=1 gpgkey=https://packages.microsoft.com/keys/microsoft.asc" | sudo tee /etc/yum.repos.d/azure-cli.repo
        !sudo dnf install azure-cli
    
    except: # pro distribuce Ubuntu, Debian 
        !curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
        

### macOS

In [ ]:
import sys

if sys.platform == "darwin":
    !brew update && brew install azure-cli

**Zdroje** ke všem druhům instalací: https://docs.microsoft.com/en-us/cli/azure/install-azure-cli

# 5. Vytvoření Machine Learning studia a Workspace

Azure Machine Learning Studio je služba zahrnující v sobě služby pro všechny části vývoje modelu. Kromě spouštění vlastních modelů stojí za zmínku možnost jej využít i pro zpracování dat, labelling nebo pro tvorbu no-code modelů skrz AutoML.

Pro práci s Machine learning studiem musíme nejdříve založit tzv. *workspace* (pracovní prostředí) který sdružuje všechny nastavení, datasety, úložiště, výpočetní zdroje nebo historii trénování jež budete používat v rámci jednoho projektu. V případě vícero projektů tak můžete mít každý workspace nastaven podle unikátních potřeb dané úlohy. 

K vytvoření workspace se potřebujeme nejprve přihlásit do Azure skrz CLI a zadat přihlašovací údaje z regisrace.

In [ ]:
!az login #Mělo by se objevit přihlašovací stránka v prohlížeči.

Nyní můžeme získat *subscription id* - unikátní klíč k našemu Azure účtu, který budeme používat pro autentizaci Python příkazů.

In [ ]:
import json

account_info = !az account show

account_info = json.loads(" ".join(list(account_info)))
sub_id = account_info["id"]

sub_id

ML Studio má pro svou obsluhu Python API v podobě balíčku, který jsme dříve nainstalovali. Budeme jej využívat i ve zbytku návodu. Pojmenujeme a vytvoříme nový workspace.

In [ ]:
from azureml.core import Workspace

ws_name = "myworkspace" 

ws = Workspace.create(name=ws_name,
                      subscription_id=sub_id,
                      resource_group='myresourcegroup',
                      create_resource_group=True,
                      location='westeu')

Abychom nemuseli pokaždé zkoušet jestli již workspace existuje, můžeme si stáhnout jeho konfigurační soubor a potom získat objekt reprezentující workspace pomocí `ws = Workspace.from_config()`

In [ ]:
ws.write_config(path="", name="config.json")

# 6. Workflow ML Studia a důležité koncepty


Při práci s ML Studiem se setkáte s pár často používanými koncepty které při vývoji zpřehledňují práci. Jedním z nich byl dříve zmíněný workspace. Pojďme si nejdříve udělat obrázek o tom jak celý proces trénování na Azuru funguje a jaké prvky jsou potřeba skrz následující diagram a jeho popis.

* **compute target**
    * Virtuální počítač v cloudu na kterém poběží výpočty.
    
    
* **control script**
    * Je spouštěný lokálně.
    * Předává hyperparametry training scriptu pomocí objektu *ScriptRunConfig*
    * Definuje trénovací prostředí skrz *Environment* (např. Python balíčky)
    * Odesílá a spouští training script na cloudu.
    

* **training script**
    * Běží v cloudu. 
    * Načítá data, definuje architekturu modelu, nastavuje hyperparametry spouští trénování a průběžně může ukládat informace. 
    * Výhodou je že se prakticky vůbec nemusí vůbec lišit od skriptu pro trénování na osobním počítači.


* **dataset**
    * lokální složka s trénovacími/validačními/testovacími daty kterou chceme do cloudu nahrát.


* **Azure Datastore**
    * Cloudové uložiště pro dataset
    * Training script spuštěn na compute target k němu může rychle přistupovat jako k lokálnímu uložišti.


* **výstupy, grafy, obrázky**
    * cokoliv můžeme během trénování ukládat přímo do ML Studia. Vše pák můžeme stáhnout a dále analyzovat lokálně.
    
Následující sekce představují detailněji tyto koncepty a jejich implementaci.

![mlstudio_workflow](img/mlstudio_workflow.png)

## 6.1 Compute Targets *(Výpočetní zdroje)*

Zjednodušeně se jedná o virtuální počítač optimalizovaný pro strojové učení v cloudu, na kterém budou probíhat veškeré výpočty. Nový *compute target* můžeme vytvořit opět pomocí balíčku `azureml`.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_target_name = 'gpu-cluster1'
    
    try:
        aml_compute = ComputeTarget(workspace=ws, name=compute_target_name)

    except ComputeTargetExhception: 

        aml_config = AmlCompute.provisioning_configuration(
            vm_size="Standard_NC6", # označení compute target
            vm_priority="dedicated", # vyhradí veškeré zdroje určitému uživateli, který tak nemusí čekat
            min_nodes = 0, # podle potřeby se může do výpočtu zapojit vícero nodes/více CPU a GPU
            max_nodes = 4,
            idle_seconds_before_scaledown=300 # pokud není část clusteru využita po této době se počet nodes zmenší
        )
        
        aml_compute = ComputeTarget.create(
            ws, 
            name=compute_target_name, 
            provisioning_configuration=aml_config
        )

        aml_compute.wait_for_completion(show_output=True)

Azure nabízí mnoho druhů compute targets, které se liší svou pamětí, počtem virtuálních CPU (vCPU), druhem GPU, ale také cenou. Pro tento notebook jsem zvolil nejlevnější typ s grafickou kartou - `Standard_NC6`. Obsahuje 6 vCPU, 56GiB RAM a jednu grafickou kartu NVIDIA Tesla K80. Hodina trénování Vás vyjde na 0.90$. Kredit studenstkého účtu tak bude zhruba stačit na 110 hodin. Seznam všech dostupných VM spolu s cenami najdete na: https://azure.microsoft.com/en-us/pricing/details/machine-learning/.

## 6.2 Environments *(Prostředí)*

Skrz definici objektu *Environment* (prostředí) můžeme konfigurovat náš compute target. V našem případě budeme volit, které Python balíčky se mají na virtuálním počítači instalovat. Můžeme to udělat několika způsoby.

### 6.2.1 Currated environment

V Azure ML Studiu je již mnoho populárních konfigurací náhráno v podobě Docker image souborů. Kvůli tomu že se nemusí balíčky stahovat, je tato varianta nejrychlejší. Detailní přehled dostupných currated prostředí a balíčků jenž obsahují najde zde: https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments.

In [ ]:
curated_env_name = 'AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu'

tf_env = Environment.get(workspace=ws, name=curated_env_name)

### 6.2.2 Vlastní environment

V případě že currated environments neobsahují některý z balíčků co potřebujeme, nabízí se možnost vytvořit si vlastní specifikaci. K vytvoření můžete použít existující virtuální prostředí Anaconda nebo soubor requirements.txt vygenerovaný skrz pip.

In [ ]:
# virtuální prostředí Anaconda
tf_env = Environment.from_existing_conda_environment(name="tf_env", conda_environment_name="<conda-env-name>")


# pip requirements.txt (pro generování v terminálu zadejte: pip freeze > requirements.txt)
myenv = Environment.from_pip_requirements(name = "tf_env", file_path="<path-to-pip-requirements-file>")                                          

Další způsoby definování prostředí: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments#use-a-curated-environment

## 6.4 ScriptRunConfig a předávání parametrů 

ScriptRunConfig zaobaluje trénovací skript a předává jej Workspace. Jak název napovídá můžeme pomocí něj nastavovat jak má náš skript běžet. Jedná se o umístění trénovacího skriptu, compute target, prostředí a argumenty trénovacího skriptu.

In [ ]:
config = ScriptRunConfig(
    source_directory='./src', # veškerý obsah adresáře src se přenese na compute target
    script='train-script.py', # soubor který se má spustit
    compute_target=aml_compute, 
    environment=tf_env,
    arguments=args
)

Typicky chceme měnit hyperparametry našeho modelu. Než přímo pokaždé měnit parametry přímo v training scriptu můžeme je před spuštěním trénování nastavit přímo v control scriptu. Před definicí ScriptRunConfig nejdříve vytvoříme list s názvy parametrů a příslušnými hodnotami. Třeba takto:

In [ ]:
args = [
    '--batch-size' , 64,
    '--learning-rate', 0.01,
    '--epochs', 3
]

Na straně training scriptu můžeme argumenty přijmou pomocí knihovny argparse

In [ ]:
import argparse

parser = argparse.ArgumentParser()

parser.add_argument('--batch-size', type=int, dest='batch_size', default=128)
parser.add_argument('--learning-rate', type=float, dest='learning_rate', default=0.01)
parser.add_argument('--epochs', type=int, dest='epochs', default=1)

args = parser.parse_args()

"""
TIP: argparse můžete použít i při spouštění skriptu lokálně v terminálu

python train-script.py --batch-size 64 --learning-rate 0.01 --epochs 3
"""

Název proměné potom definuje atribut `dest`. Náš parametr uvnitř training scriptu pak dostaneme jako `args.batch_size`

## 6.5 Objekt Run - ukládání metrik a tagy

Instance objektu *Run* reprezentuje pokus neboli jedno spuštění trénování v rámci *experimentu* (viz níže). Můžeme ho použít pro asynchroní monitorování tréninku, ukládání metrik nebo jakýchkoliv jiných výstupů v training scriptu. Uvádím několik příkladů použití:

In [ ]:
from azureml.core import Run

run = Run.get_context() # získání instance

# skalár
run.log("accuracy", 0.42)

# list
run.log_list("accuracies", [0.31, 0.41, 0.59])

# dictionary
run.log_table("acc table", {"ep1" : [0.27, 0.18], "ep2" : [0.28, 0.18]})

# obrázek
run.log_image("ROC", path=./roc_curve")

Prostor pro metriky s danými jmény se vytvoří automaticky. 

Za zmínku ještě stojí možnost přidávat k jednotlivým pokusům tzv. **tagy**. Ty nám dovolí jednotlivé pokusy okomentovat nebo označit a usnadnit nám na konci návodu hledání v jejich seznamu. Pro přidání tagu zavoláme v training scriptu run.tag(\<nazev tagu>, \<hodnota>)

Další využití objektu Run: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run.run?view=azure-ml-py#azureml_core_run_Run_log_table




## 6.3 Experiments

Objekt *Experiment* můžete použít k seskupení několika pokusů týkajících se ověření jedné hypotézy. Například pro testování výkonu konkrétní architektury. Založit environment a získat objekt pro jeho obsluhu můžeme takto: (Pokud environment se zadaným jménem již existuje, vrátí objekt existujícího prostředí.)

In [ ]:
experiment = Experiment(workspace=ws, name='cifar10-conv-test1')

# 7. Upload datasetu do Azure Datastore

Velikost datasetů může činit až několik GB a proto nedává smysl při každém spuštění trénování uploadovat i celý data set. Pokud jednorázově dataset nahrajeme na cloudové uložiště trénovací skript má data vždy u sebe, a my tak můžeme rychleji iterovat.

Pro demonstraci budu model trénovat na datasetu populárním datasetu CIFAR-10, jenž obsahuje 60000 32x32, patřících do 10 tříd. Celý dataset pak zabírá 163 MB.

Dataset můžeme jednoduše stáhnout do místní složky pomocí knihovny `tensorflow_datasets`.

In [ ]:
import tensorflow_datasets as tfds

tfds.load(name="cifar10", split=["train", "test"], data_dir=".", download=True, as_supervised=True, batch_size=-1)

<img src="img/cifar10_classes.jpg" alt="cifar10_classes" style="width: 600px;"/>

Nyní jak nahrát tento dataset do Cloudu. Azure Datastore vlastně není úplně uložiště samo o sobě. Pouze ukládá připojovací údaje ke klasickým Azure uložištím jako Blob Container, Data Lake, SQL Database etc. Jedná se tak o užitečnou abstrakci, která nám umožní se v trénovacím skriptu k uložišti připojit a jednat s ním jako s lokálním. 

Každý založený workspace už svůj defaultní datastore má, a proto nebudeme vytvářet nový. K uploadu nám stačí pouze specifikovat relativní cestu ke složce obsahující dataset spolu s cestou pod kterou později složku s datasetem najdeme při připojení na cloudové uložiště v training scriptu. 

In [ ]:
datastore = ws.get_default_datastore()

datastore.upload(src_dir='./cifar10',
                 target_path='datasets/cifar10',
                 overwrite=True) # Pokud na uložišti soubor s tímto názvem existuje přepíše se.

V control scriptu poté můžeme získat referenci na uložiště. 

In [ ]:
data_ref = datastore.path('datasets').as_mount()

Aby mohl training script přistupovat k úložišti jako k lokálnímu musíme

1. referenci předat skrz argumenty v ScriptConfig jako `str(data_ref)`
2. nastavit její název jako `config.run_config.data_references = {data_ref.data_reference_name : data_ref.to_config()}`

(konkrétně ukázáno níže)



# 8. Dáváme to dohromady

Teď známe vše potřebné abychom mohli začít s trénováním. Následují detailně okomentované soubory control-script.py a training-script.py, které obsahují již v předchozích sekcích zmíněné prvky.

Oba soubory jsou přiloženy k tomuto notebooku. Můžete si je později upravit pro vlastní účely.

## 8.1 Control script

In [21]:
from pygments import highlight; from pygments.lexers import PythonLexer; from pygments.formatters import HtmlFormatter; import IPython;
with open('control-script.py') as f: code = f.read(); formatter = HtmlFormatter(); 
IPython.display.HTML('<style type="text/css">{}</style>{}'.format(formatter.get_style_defs('.highlight'),highlight(code, PythonLexer(), formatter)))

### 8.2 Training script

In [13]:
with open('src/training-script.py') as f: code = f.read(); formatter = HtmlFormatter()
IPython.display.HTML('<style type="text/css">{}</style>{}'.format(formatter.get_style_defs('.highlight'),highlight(code, PythonLexer(), formatter)))

# 9. Spuštění a monitoring trénování

Pro počátek trénování spustíme control script. Ten zašle požadavek o trénování na Azure kde se podle našeho Environment objektu vytvoří prostředí s potřebnými balíčky.  

In [ ]:
!python control-script.py

Control script na konci do konzole vrátí odkaz na Azure Portal, kde můžete v grafické podobě sledovat průběh trénování. Můžeme v reálném čase sledovat automaticky se tvořící se grafy našich metrik nebo sledovat klasický výstup Python konzole.

<img src="img/azportal_metrics.png" alt="Drawing" style="width: 900px;"/>

<img src="img/azportal_logs.png" alt="Drawing" style="width: 900px;"/>

# 10. Přístup k předchozím experimentům

Po natrénování modelu ho většinou chceme dostat zpět do lokálního zařízení a nasledně ho otestovat nebo aplikovat. Training script ukazuje že můžeme model uložit již známým způsobem (uložení do složky `./outputs` je nutné). Limit úložiště jednoho pokusu je 300MB což by mělo pro většinu modelů a metrik stačit (jinak se nabízí ukládání do Azure Datastore).

Nejdříve musíme získat Run objekt pro pokus, jehož výstup chceme stáhnout. Můžeme ho najít v kompletní historii všech trénování v rámci jednoho experimentu. Pro orientaci můžeme použít filtrování za pomocí tagů.

In [ ]:
exp = Experiment(workspace=ws, name='cifar10-conv-test1')

experiment.get_runs() # získá všechny pokusy

#tag = tags={"run number" : 3} # zobrazí se pouze trénování s daným tagem
tag = None # všechny

runs = list(experiment.get_runs(tags=tag))

for i, run in enumerate(runs):
    print(i)
    print(run)
    print(run.get_portal_url())
    #print(run.get_tags()) # zobrazí všechny tagy trénování
    print("\n\n")

Následně můžeme zvolit jeden z pokusů pomocí indexu, a stáhnout potřebné soubory ze složky `./outputs`. Stahovat a ukládat taky můžete do složky `./logs`.

In [ ]:
run = runs[0]

run.download_file("./outputs/trained_model.h5")

# run.download_files(./outputs) # stáhné veškeré uložené výstupy pokusu

Pokud například chcete **pokračovat v trénování již staženého modelu**, stačí pouze jeho soubor přidat so složky `./src` a v training scriptu jej použít.

# Závěr

Při tvorbě návodu jsem se snažil zmínit jenom to opravdu nejpodstatnější. Informace obsažené v dokumentaci mohou být totiž pro většinu aplikací nadbytečné. Pokud byste si ale chtěli o některém z témat přečíst více nebo využijte odkazy na dokumentaci které jsem v průběhu příručky přikládal.

Pro vlastní trénování bude určitě potřeba modifikovat oba skripty. V případě chyb, nepřesností, ale i podnětů na zlepšení neváhejte přispět pod GitHub repozitář: 